# <font size="6"><b>Assignment V: GitHub and the ticketmaster.com API (Python)</b></font>
<font size="5">Data Science Project Management (DS400) | Winter Term 2022/23</font>

<br>

<div class="author_year">
Submitted by Jakob Zgonc (6293178)<br>
Submitted on 23.01.2023
</div>

<br>

---

<h2>Table of Contents<span class="tocSkip"></span></h2>

<div class="toc">
    <ul class="toc-item", style="list-style-type:none;">
        <li><span><a href="#1-Interacting-with-the-API---the-basics">1 Interacting with the API - the basics</a></span></li>
        <li><span><a href="#2-Interacting-with-the-API---advanced">2 Interacting with the API - advanced</a></span></li>
        <li><span><a href="#3-Visualizing-the-extracted-data">3 Visualizing the extracted data</a></span></li>
        <li><span><a href="#4-Event-locations-in-other-countries">4 Event locations in other countries</a></span></li>
    </ul>
</div>

<br>

---

**Code of conduct**

*I hereby acknowledge that the submitted assignment is my own work. During the preparation of this assignment I have worked together with Max Mohr and Felix Koehn.*

<br>

---

In [1]:
%%html
<style>

div.author_year {
    color: #708090;
    font-size: 17px;
    font-style: italic;
}

div.task {
    background-color:#DAE3F3;
    color: #337AB7; /*  #002060;*/
    border-radius: 10px; 
    padding: 20px;
    margin-top: 20px;
    margin-bottom: 20px;
    margin-left: -5px;
    margin-right: -20px;
    font-size: 15px;
    border-left: 10px solid #337AB7;
}

body {
    max-width:960px;
    margin:0 auto;
}

</style>

This document is also available in [this GitHub repository](https://github.com/jzgonc/dspm_2022_assignment_5).

---

## 1 Interacting with the API - the basics

<div class="task">
    <ol start=7>
        <li>Perform a first <code>GET</code> request, that searches for event venues in Germany (<code>countryCode = "DE"</code>). Extract the content from the response object and inspect the resulting list. Describe what you can see.
        </li>
        </ol>
</div>

In [2]:
# import packages
import requests
import numpy as np
import pandas as pd
import time

In [3]:
# get API key
with open('ticketmaster_api_key.txt','r') as file:
    apikey = file.read()

In [4]:
# url for venues
url = 'https://app.ticketmaster.com/discovery/v2/venues'

In [5]:
# define params for API call
params = dict(apikey=apikey, countryCode = 'DE')

# get response from API call as json (i.e. dictionary)
venues_DE_first_page = requests.get(url=url, params=params).json()

# show response
venues_DE_first_page

{'_embedded': {'venues': [{'name': 'Gruenspan',
    'type': 'venue',
    'id': 'KovZpZAneakA',
    'test': False,
    'url': 'http://www.ticketmaster.de/venue/287155',
    'locale': 'en-de',
    'images': [{'ratio': '16_9',
      'url': 'https://s1.ticketm.net/dbimages/2057v.',
      'width': 205,
      'height': 115,
      'fallback': False}],
    'postalCode': '22767',
    'timezone': 'Europe/Berlin',
    'city': {'name': 'Hamburg'},
    'country': {'name': 'Germany', 'countryCode': 'DE'},
    'address': {'line1': 'Grosse Freiheit 58'},
    'location': {'longitude': '9.958075', 'latitude': '53.551885'},
    'markets': [{'name': 'Germany', 'id': '210'}],
    'dmas': [{'id': 610}],
    'boxOfficeInfo': {'phoneNumberDetail': 'Gruenspan Große Freiheit 58 22767 Hamburg Tel: 040-313616 mail: info@gruenspan.de web: www.gruenspan.de'},
    'upcomingEvents': {'_total': 2, 'mfx-de': 2, '_filtered': 0},
    'ada': {'adaPhones': '+49.(0)1805 - 969 0000 (14 Ct./Min.)',
     'adaCustomCopy': 'Soll

The response of the API call is a nested dictionary (not a list as stated in the task, I hope that's fine anyway). On the first level there are three items:
* `_embedded` contains the actual data we have requested. The value is a dictionary itself. In this dictionary, again, there is a list with all returned venues (each of which is a dictionary, again)
* `_links` contains the url that was used for the API call (excluding the apikey and the base url)
* `page` contains information about the returned page from the data source

<div class="task">
    <ol start=8>
        <li>Extract the <code>name</code>, the <code>city</code>, the <code>postalCode</code> and <code>address</code>, as well as the <code>url</code> and the <code>longitude</code> and <code>latitude</code> of the <code>venues</code> to a data frame. This data frame should have the following structure:
        <br><br>
<code>## Rows: 20
## Columns: 7
## $ name       chr "Gruenspan", "Huxleys Neue Welt", "Kleine Olympiahalle", "Z~
## $ city       chr "Hamburg", "Berlin", "Munich", "Emmelshausen", "Mülheim", "~
## $ postalCode dbl 22767, 10967, 80809, 56281, 45479, 76646, 68766, 44263, 542~
## $ address    chr "Grosse Freiheit 58", "Hasenheide 107 – 113", "Spiridon-Lou~
## $ url        chr "http://www.ticketmaster.de/venue/287155", "http://www.tick~
## $ longitude  dbl 9.958075, 13.421380, 11.550920, 7.556560, 6.874710, 8.59908~
## $ latitude   dbl 53.55188, 52.48639, 48.17543, 50.15544, 51.42778, 49.12692,~</code></li>
</ol>
</div>

## 2 Interacting with the API - advanced

<div class="task">
    <ol start=9>
        <li>Have a closer look at the list element named <code>page</code>. Did your <code>GET</code> request from exercise (7) return <i>all</i> event locations in Germany? Obviously not - there are of course much more venues in Germany than those contained in this list. Your <code>GET</code> request only yielded the first results page containing the first 20 out of several thousands of venues. Check the API documentation under the section <a href="https://developer.ticketmaster.com/products-and-docs/apis/discovery-api/v2/#search-venues-v2">Venue Search</a>. How can you request the venues from the remaining results pages? Iterate over the results pages and perform <code>GET</code> requests for all venues in Germany. After each iteration, extract the seven variables <code>name</code>, <code>city</code>, <code>postalCode</code>, <code>address</code>, <code>url</code>, <code>longitude</code>, and <code>latitude</code>. Join the information in one large data frame. Print the first 10 rows and the shape of the resulting data frame. The resulting data frame should look something like this (note that the exact number of search results may have changed since this document has been last modified):<br><br>
<code>## Rows: 12,671
## Columns: 7
## $ name       chr "Gruenspan", "Huxleys Neue Welt", "Kleine Olympiahalle", "Z~
## $ city       chr "Hamburg", "Berlin", "Munich", "Emmelshausen", "Mülheim", "~
## $ postalCode dbl 22767, 10967, 80809, 56281, 45479, 76646, 68766, 44263, 542~
## $ address    chr "Grosse Freiheit 58", "Hasenheide 107 – 113", "Spiridon-Lou~
## $ url        chr "http://www.ticketmaster.de/venue/287155", "http://www.tick~
## $ longitude  dbl 9.958075, 13.421380, 11.550920, 7.556560, 6.874710, 8.59908~
## $ latitude   dbl 53.55188, 52.48639, 48.17543, 50.15544, 51.42778, 49.12692,~</code></li>
</ol>
</div>

## 3 Visualizing the extracted data

<div class="task">
    <ol start=10>
        <li>Below, you can find code that produces a map of Germany. Add points to the map indicating the locations of the event venues across Germany.</li>
    </ol>
</div>

<div class="task">
    <ol start=11>
        <li>You will find that some coordinates lie way beyond the German borders and can be assumed to be faulty. Set coordinate values to <code>NA</code> where the value of <code>longitude</code> is outside the range (<code>5.866, 15.042</code>) or where the value of <code>latitude</code> is outside the range (<code>47.270, 55.059</code>) (these coordinate ranges have been derived from the extreme points of Germany as listed on Wikipedia (see <a href="https://en.wikipedia.org/wiki/Geography_of_Germany#Extreme_points">here</a>). For extreme points of other countries, see <a href="https://en.wikipedia.org/wiki/Lists_of_extreme_points#Sovereign_states">here</a>).</li>
        </ol>
</div>

## 4 Event locations in other countries

<div class="task">
    <ol start=12>
        <li>Repeat exercises (9)–(11) for another European country of your choice. (Hint: Clean code pays off! If you have coded the exercises efficiently, only very few adaptions need to be made.)</li>
    </ol>
</div>


<br>

***